# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 27 2023 8:30AM,256032,10,MSP217957,"Methapharm, Inc.",Released
1,Jan 27 2023 8:30AM,256032,10,MSP217958,"Methapharm, Inc.",Released
2,Jan 27 2023 8:30AM,256032,10,MSP217959,"Methapharm, Inc.",Released
3,Jan 27 2023 8:30AM,256032,10,MSP217960,"Methapharm, Inc.",Released
4,Jan 27 2023 8:30AM,256032,10,MSP217961,"Methapharm, Inc.",Released
5,Jan 27 2023 8:30AM,256032,10,MSP217962,"Methapharm, Inc.",Released
6,Jan 27 2023 8:30AM,256032,10,MSP217963,"Methapharm, Inc.",Released
7,Jan 27 2023 8:30AM,256039,10,0086333238,ISDIN Corporation,Released
8,Jan 27 2023 8:30AM,256040,10,0086333240,ISDIN Corporation,Released
9,Jan 27 2023 8:30AM,256039,10,0086333239,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,256035,Released,6
26,256038,Released,1
27,256039,Released,19
28,256040,Released,5
29,256041,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256035,NaN,NaN,6.0
256038,NaN,NaN,1.0
256039,NaN,NaN,19.0
256040,NaN,NaN,5.0
256041,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255953,44.0,12.0,1.0
255969,0.0,1.0,0.0
255987,11.0,18.0,20.0
255994,9.0,10.0,2.0
255997,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255953,44,12,1
255969,0,1,0
255987,11,18,20
255994,9,10,2
255997,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255953,44,12,1
1,255969,0,1,0
2,255987,11,18,20
3,255994,9,10,2
4,255997,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255953,44,12,1
1,255969,,1,
2,255987,11,18,20
3,255994,9,10,2
4,255997,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 27 2023 8:30AM,256032,10,"Methapharm, Inc."
7,Jan 27 2023 8:30AM,256039,10,ISDIN Corporation
8,Jan 27 2023 8:30AM,256040,10,ISDIN Corporation
10,Jan 27 2023 8:30AM,256035,10,ISDIN Corporation
37,Jan 27 2023 8:30AM,256033,10,Emerginnova
38,Jan 26 2023 4:00PM,256041,19,"Methapharm, Inc."
39,Jan 26 2023 3:43PM,256038,12,HealthCaps LLC
40,Jan 26 2023 3:35PM,256034,16,Sartorius Bioprocess Solutions
41,Jan 26 2023 3:25PM,256031,21,"NBTY Global, Inc."
42,Jan 26 2023 3:14PM,256030,16,TASA USA Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 27 2023 8:30AM,256032,10,"Methapharm, Inc.",,,7
1,Jan 27 2023 8:30AM,256039,10,ISDIN Corporation,,,19
2,Jan 27 2023 8:30AM,256040,10,ISDIN Corporation,,,5
3,Jan 27 2023 8:30AM,256035,10,ISDIN Corporation,,,6
4,Jan 27 2023 8:30AM,256033,10,Emerginnova,,,1
5,Jan 26 2023 4:00PM,256041,19,"Methapharm, Inc.",,,1
6,Jan 26 2023 3:43PM,256038,12,HealthCaps LLC,,,1
7,Jan 26 2023 3:35PM,256034,16,Sartorius Bioprocess Solutions,,,1
8,Jan 26 2023 3:25PM,256031,21,"NBTY Global, Inc.",,,1
9,Jan 26 2023 3:14PM,256030,16,TASA USA Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 8:30AM,256032,10,"Methapharm, Inc.",7,,
1,Jan 27 2023 8:30AM,256039,10,ISDIN Corporation,19,,
2,Jan 27 2023 8:30AM,256040,10,ISDIN Corporation,5,,
3,Jan 27 2023 8:30AM,256035,10,ISDIN Corporation,6,,
4,Jan 27 2023 8:30AM,256033,10,Emerginnova,1,,
5,Jan 26 2023 4:00PM,256041,19,"Methapharm, Inc.",1,,
6,Jan 26 2023 3:43PM,256038,12,HealthCaps LLC,1,,
7,Jan 26 2023 3:35PM,256034,16,Sartorius Bioprocess Solutions,1,,
8,Jan 26 2023 3:25PM,256031,21,"NBTY Global, Inc.",1,,
9,Jan 26 2023 3:14PM,256030,16,TASA USA Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 8:30AM,256032,10,"Methapharm, Inc.",7,,
1,Jan 27 2023 8:30AM,256039,10,ISDIN Corporation,19,,
2,Jan 27 2023 8:30AM,256040,10,ISDIN Corporation,5,,
3,Jan 27 2023 8:30AM,256035,10,ISDIN Corporation,6,,
4,Jan 27 2023 8:30AM,256033,10,Emerginnova,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 8:30AM,256032,10,"Methapharm, Inc.",7.0,NaN,NaN
1,Jan 27 2023 8:30AM,256039,10,ISDIN Corporation,19.0,NaN,NaN
2,Jan 27 2023 8:30AM,256040,10,ISDIN Corporation,5.0,NaN,NaN
3,Jan 27 2023 8:30AM,256035,10,ISDIN Corporation,6.0,NaN,NaN
4,Jan 27 2023 8:30AM,256033,10,Emerginnova,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 8:30AM,256032,10,"Methapharm, Inc.",7.0,0.0,0.0
1,Jan 27 2023 8:30AM,256039,10,ISDIN Corporation,19.0,0.0,0.0
2,Jan 27 2023 8:30AM,256040,10,ISDIN Corporation,5.0,0.0,0.0
3,Jan 27 2023 8:30AM,256035,10,ISDIN Corporation,6.0,0.0,0.0
4,Jan 27 2023 8:30AM,256033,10,Emerginnova,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1792195,44.0,12.0,9.0
12,256038,1.0,0.0,0.0
15,1024014,28.0,18.0,11.0
16,1024121,4.0,0.0,0.0
19,1024018,4.0,12.0,44.0
21,768004,1.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1792195,44.0,12.0,9.0
1,12,256038,1.0,0.0,0.0
2,15,1024014,28.0,18.0,11.0
3,16,1024121,4.0,0.0,0.0
4,19,1024018,4.0,12.0,44.0
5,21,768004,1.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,44.0,12.0,9.0
1,12,1.0,0.0,0.0
2,15,28.0,18.0,11.0
3,16,4.0,0.0,0.0
4,19,4.0,12.0,44.0
5,21,1.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,44.0
1,12,Released,1.0
2,15,Released,28.0
3,16,Released,4.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Completed,9.0,0.0,11.0,0.0,44.0,0.0
Executing,12.0,0.0,18.0,0.0,12.0,2.0
Released,44.0,1.0,28.0,4.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Completed,9.0,0.0,11.0,0.0,44.0,0.0
1,Executing,12.0,0.0,18.0,0.0,12.0,2.0
2,Released,44.0,1.0,28.0,4.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Completed,9.0,0.0,11.0,0.0,44.0,0.0
1,Executing,12.0,0.0,18.0,0.0,12.0,2.0
2,Released,44.0,1.0,28.0,4.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()